<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import csv as csv

In [2]:
# Load training data
train_df = pd.read_csv("data/train.csv", header=0)

In [3]:
train_df.head(3)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
train_label = train_df.values[:,0]
train_label

array([1, 0, 1, ..., 7, 6, 9])

In [5]:
from tensorflow.python.keras.utils import to_categorical
train_label = to_categorical(train_label)
train_label

/home/kONODERA/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/kONODERA/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [6]:
train_data = train_df.values[:,1:]
train_data.reshape(42000,28,28)
train_data = train_data/255.

In [7]:
train_data = train_data.reshape(-1,28,28,1)
train_data.shape

(42000, 28, 28, 1)

In [8]:
# Load testning data
test_df = pd.read_csv("data/test.csv", header=0)
test_df.head(3)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
test_data = test_df.values
test_data.reshape(28000,28,28)
test_data = test_data/255.

In [10]:
test_data = test_data.reshape(-1,28,28,1)
test_data.shape

(28000, 28, 28, 1)

In [11]:
# とりあえず先頭の20000のみを使用
#train_data = train_data[:20000]
#train_label = train_label[:20000]

In [12]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D,MaxPooling2D,Dropout,Flatten,Dense
model = Sequential()

In [13]:
def latest_model():
    model.add(
        Conv2D(
            filters=32,
            input_shape=(28, 28, 1),
            kernel_size=(3, 3),
            strides=(1, 1),
            padding='same',
            activation='relu'
        )
    )

    model.add(
        Conv2D(
            filters=32,
            kernel_size=(3, 3),
            strides=(1, 1),
            padding='same',
            activation='relu'
        )
    )
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Dropout(0.25))
    
    model.add(
        Conv2D(
            filters=64,
            kernel_size=(3, 3),
            strides=(1, 1),
            padding='same',
            activation='relu'
        )
    )
    
    model.add(
        Conv2D(
            filters=64,
            kernel_size=(3, 3),
            strides=(1, 1),
            padding='same',
            activation='relu'
        )
    )
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    
    model.add(Dense(units=512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=10, activation='softmax'))

In [14]:
latest_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
__________

In [15]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_model1 = model.fit(
    train_data,
    train_label,
    batch_size=32,
    epochs=5,
    validation_split=0.2
)

Train on 33600 samples, validate on 8400 samples
Epoch 1/5
33600/33600 [==============================] - 13s - loss: 0.2002 - acc: 0.9368 - val_loss: 0.0631 - val_acc: 0.9814
Epoch 2/5
33600/33600 [==============================] - 8s - loss: 0.0735 - acc: 0.9776 - val_loss: 0.0415 - val_acc: 0.9870
Epoch 3/5
33600/33600 [==============================] - 8s - loss: 0.0525 - acc: 0.9845 - val_loss: 0.0575 - val_acc: 0.9831
Epoch 4/5
33600/33600 [==============================] - 8s - loss: 0.0465 - acc: 0.9850 - val_loss: 0.0371 - val_acc: 0.9900
Epoch 5/5
33600/33600 [==============================] - 8s - loss: 0.0384 - acc: 0.9877 - val_loss: 0.0317 - val_acc: 0.9901


In [16]:
res = model.predict(test_data)

In [17]:
res[0]

array([5.7167449e-09, 4.8670965e-09, 9.9999976e-01, 2.1033276e-07,
       2.1935422e-09, 5.6038580e-11, 3.5604139e-10, 3.1710712e-08,
       3.7139468e-08, 8.2448791e-11], dtype=float32)

In [18]:
output = [0 for i in range(len(res))]
ids = [i for i in range(1,len(res)+1)]
for i in range(len(res)):
    output[i] = res[i].argmax()

In [19]:
submit_file = open("submit.csv", "w")
file_object = csv.writer(submit_file)
file_object.writerow(["ImageId", "Label"])
file_object.writerows(zip(ids, output))
submit_file.close()